In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import os
import gc
import xgboost as xgb
from xgboost import plot_importance
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import dask
import dask.dataframe as dd   
#!pip install dask_ml
#from dask_ml.model_selection import train_test_split
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')



/kaggle/input/adtrackingfraud-lightgbm/__results__.html
/kaggle/input/adtrackingfraud-lightgbm/__notebook__.ipynb
/kaggle/input/adtrackingfraud-lightgbm/__output__.json
/kaggle/input/adtrackingfraud-lightgbm/custom.css
/kaggle/input/adtrackingfraud-lightgbm/train-ip-devices-os-and-hour-clicks-downloads-dwratio.csv/009.part
/kaggle/input/adtrackingfraud-lightgbm/train-ip-devices-os-and-hour-clicks-downloads-dwratio.csv/081.part
/kaggle/input/adtrackingfraud-lightgbm/train-ip-devices-os-and-hour-clicks-downloads-dwratio.csv/051.part
/kaggle/input/adtrackingfraud-lightgbm/train-ip-devices-os-and-hour-clicks-downloads-dwratio.csv/048.part
/kaggle/input/adtrackingfraud-lightgbm/train-ip-devices-os-and-hour-clicks-downloads-dwratio.csv/052.part
/kaggle/input/adtrackingfraud-lightgbm/train-ip-devices-os-and-hour-clicks-downloads-dwratio.csv/092.part
/kaggle/input/adtrackingfraud-lightgbm/train-ip-devices-os-and-hour-clicks-downloads-dwratio.csv/055.part
/kaggle/input/adtrackingfraud-lightgbm/

In [3]:
dtypes = {'ip':'uint32',
          'app': 'uint16',
          'device': 'uint16',
          'os': 'uint16',
          'channel': 'uint16',
          'is_attributed': 'uint8'}

params = {'eta': 0.3,
          'tree_method': "hist",
          'grow_policy': "lossguide",
          'max_leaves': 1400,  
          'max_depth': 0, 
          'subsample': 0.9, 
          'colsample_bytree': 0.7, 
          'colsample_bylevel':0.7,
          'min_child_weight':0,
          'alpha':4,
          'objective': 'binary:logistic', 
          'scale_pos_weight':9,
          'eval_metric': 'auc', 
          'nthread':8,
          'random_state': 99, 
          'silent': True} 
class xg_generator():
    def __init__(self, dtypes = dtypes, params = params):
        self.params = params
        self.dtypes = dtypes
    def __load__(self):
        Xtrain_concates = []
        ytrain_concates = []
        Xval_concates = []
        yval_concates = []
        for idx in range(115):#(118):
            train_dir = "/kaggle/input/adtrackingfraud-lightgbm/train-ip-devices-os-and-hour-clicks-downloads-dwratio.csv/{0:03}.part".format(idx)
            train = pd.read_csv(train_dir, dtype = self.dtypes, parse_dates = ['click_time', 'attributed_time'])
            
            tr_cols = list(range(5)) + list(range(8, 15)) 
            X = train.values[:, tr_cols]
            y = train.values[:, 7]
            X_train, X_val, y_train, y_val = train_test_split(X, y, random_state = 369)
            
            # X_train.concat(X_train, ignore_index=True)
            
#             dtrain = xgb.DMatrix(X_train, y_train)
#             dvalid = xgb.DMatrix(X_val , y_val)
            
            
            Xtrain_concates.append(X_train)
            Xval_concates.append(X_val)
            ytrain_concates.append(y_train)
            yval_concates.append(y_val)
            
            if (idx + 1) % 5:
                X_train = np.vstack(Xtrain_concates)
                X_val = np.vstack(Xval_concates)
                y_train = np.hstack(ytrain_concates)
                y_val = np.hstack(yval_concates)
                
                dtrain = xgb.DMatrix(X_train, y_train)
                dvalid = xgb.DMatrix(X_val , y_val)
                
                watchlist = [(dtrain, 'train'), (dvalid, 'valid')]
                Xtrain_concates = []
                ytrain_concates = []
                Xval_concates = []
                yval_concates = []
            if idx == 0:
                model = xgb.train(self.params, dtrain, 30, watchlist, early_stopping_rounds=60, maximize=True, verbose_eval=5)
                self.X_val0 = X_val
                self.y_val0 = y_val
            else:
                model = xgb.train(self.params, dtrain, 30, watchlist, early_stopping_rounds=60, maximize=True, verbose_eval=5, xgb_model='xgmodel.ml')
            model.save_model('xgmodel.ml')




In [ ]:
# X_val0, y_val0 to see what happended
X_val0 = g.X_val0
y_val0 = g.y_val0
dvalid0 = xgb.DMatrix(X_val0, y_val0)

In [ ]:
g = xg_generator(dtypes)
g.__load__()

[05:47:15] WARNING: ../src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.89063	valid-auc:0.88218
[5]	train-auc:0.97927	valid-auc:0.97698
[10]	train-auc:0.99109	valid-auc:0.98565
[15]	train-auc:0.99788	valid-auc:0.98588
[20]	train-auc:0.99971	valid-auc:0.98652
[25]	train-auc:0.99980	valid-auc:0.98661
[29]	train-auc:0.99982	valid-auc:0.98685
[05:48:18] WARNING: ../src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.99983	valid-auc:0.98680
[5]	train-auc:0.99984	v

In [ ]:
X_val0 = g.X_val0
y_val0 = g.y_val0
dvalid0 = xgb.DMatrix(X_val0, y_val0)
watchlist = [(dvalid0, 'train'), (dvalid0, 'valid')]
xgb_model_latest = xgb.XGBClassifier() # or which ever sklearn booster you're are using

xgb_model_latest.load_model('xgmodel.ml')
#model = xgb.train(params, dvalid0, 0, watchlist, early_stopping_rounds=0, maximize=True, verbose_eval=5, xgb_model='xgmodel.ml')

In [ ]:
predict_val = xgb_model_latest.predict(g.X_val0)

In [ ]:
a = [i for i in y_val0]


In [ ]:
len(a) + len(b)

In [ ]:
len(y_val0)

In [ ]:
type(y_val0)

In [ ]:
type(predict_val)

In [ ]:
from sklearn.metrics import roc_auc_score
auc_score = roc_auc_score(a, predict_val)

In [ ]:
auc_score

In [ ]:
train_dir = "/kaggle/input/adtrackingfraud-lightgbm/train-ip-devices-os-and-hour-clicks-downloads-dwratio.csv/{0:03}.part".format(idx)
            train = pd.read_csv(train_dir, dtype = self.dtypes, parse_dates = ['click_time', 'attributed_time'])
            
tr_cols = list(range(5)) + list(range(8, 15)) 
X = train.values[:, tr_cols]
y = train.values[:, 7]
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state = 369)
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]


model = xgb.train(params, dtrain, 30, watchlist, early_stopping_rounds=60, maximize=True, verbose_eval=5)

In [ ]:
def generater(): # 000 -- 117
    for idx in range(118):
        train_dir = "/kaggle/input/adtrackingfraud-lightgbm/train-ip-devices-os-and-hour-clicks-downloads-dwratio.csv/{}.part".format(idx)
        train = pd.read_csv(train_dir, dtype = dtypes, parse_dates=['click_time', 'attributed_time'])



    
        
#raise stopiteration


In [ ]:



train = pd.read_csv(train_dir, dtype=dtypes, parse_dates=['click_time', 'attributed_time'])
train.head()

In [ ]:

tr_cols = list(range(5)) + list(range(8, 15))
X = train.values[:, tr_cols]
y = train.values[:, 7]
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state = 367)

In [ ]:
train_dir3 = "/kaggle/input/adtrackingfraud-lightgbm/train-ip-devices-os-and-hour-clicks-downloads-dwratio.csv/002.part"
train = pd.read_csv(train_dir3, dtype=dtypes, parse_dates=['click_time', 'attributed_time'])
tr_cols = list(range(5)) + list(range(8, 15))
X = train.values[:, tr_cols]
y = train.values[:, 7]
X_train3, X_val3, y_train3, y_val3 = train_test_split(X, y, random_state = 367)

In [ ]:
# 2631/1568734: yes/No

In [ ]:
# cluster = dask.distributed.LocalCluster(n_workers=4, threads_per_worker=1)
# client = dask.distributed.Client(cluster)

# dtrain = xgb.dask.DaskDMatrix(client, X_train, y_train)

# output = xgb.dask.train(client,
#                         {'verbosity': 2,
#                          'tree_method': 'hist',
#                          'objective': 'reg:squarederror'
#                          },
#                         dtrain,
#                         num_boost_round=4, evals=[(dtrain, 'train')])

In [ ]:


dtypes = {'ip':'uint32',
          'app': 'uint16',
          'device': 'uint16',
          'os': 'uint16',
          'channel': 'uint16',
          'is_attributed': 'uint8'}

params = {'eta': 0.3,
          'tree_method': "hist",
          'grow_policy': "lossguide",
          'max_leaves': 1400,  
          'max_depth': 0, 
          'subsample': 0.9, 
          'colsample_bytree': 0.7, 
          'colsample_bylevel':0.7,
          'min_child_weight':0,
          'alpha':4,
          'objective': 'binary:logistic', 
          'scale_pos_weight':9,
          'eval_metric': 'auc', 
          'nthread':8,
          'random_state': 99, 
          'silent': True} 
train_dir = "/kaggle/input/adtrackingfraud-lightgbm/train-ip-devices-os-and-hour-clicks-downloads-dwratio.csv/000.part"
train = pd.read_csv(train_dir, dtype=dtypes, parse_dates=['click_time', 'attributed_time'])

tr_cols = list(range(5)) + list(range(8, 15))
X = train.values[:, tr_cols]
y = train.values[:, 7]
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state = 367)


dtrain = xgb.DMatrix(X_train, y_train)
dvalid = xgb.DMatrix(X_val , y_val )
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]
model = xgb.train(params, dtrain, 200, watchlist, maximize=True, early_stopping_rounds = 25, verbose_eval=5)




In [ ]:
pred02 = model.predict(dvalid)

In [ ]:
pred020 = [1 if i > 0.5 else 0 for i in pred02]

In [ ]:
from sklearn.metrics import roc_auc_score
auc_score = roc_auc_score(pred020, y_val)

In [ ]:
auc_score

In [ ]:
X_train, X_test, y_train, y_test = ttsplit(X, y, test_size=0.1, random_state=0)
X_train_1, X_train_2, y_train_1, y_train_2 = ttsplit(X_train, 
                                                     y_train, 
                                                     test_size=0.5,
                                                     random_state=0)

xg_train_1 = xgb.DMatrix(X_train_1, label=y_train_1)
xg_train_2 = xgb.DMatrix(X_train_2, label=y_train_2)
xg_test = xgb.DMatrix(X_test, label=y_test)

params = {'objective': 'reg:linear', 'verbose': False}
model_1 = xgb.train(params, xg_train_1, 30)
model_1.save_model('model_1.model')

# ================= train two versions of the model =====================#
model_2_v1 = xgb.train(params, xg_train_2, 30)
model_2_v2 = xgb.train(params, xg_train_2, 30, xgb_model='model_1.model')

print(mse(model_1.predict(xg_test), y_test))     # benchmark
print(mse(model_2_v1.predict(xg_test), y_test))  # "before"
print(mse(model_2_v2.predict(xg_test), y_test))  # "after"

In [ ]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier(         learning_rate =0.1, n_estimators=140, max_depth=5,
min_child_weight=1, gamma=0, subsample=0.8,             colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4,     scale_pos_weight=1, seed=27), 
 param_grid = param_test1,     scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(train[predictors],train[target])
gsearch1.grid_scores_, gsearch1.best_params_,     gsearch1.best_score_

In [ ]:
test_dir = "/kaggle/input/talkingdata-adtracking-fraud-detection/train_sample.csv"
dtypes = {'ip':'uint32',
          'app': 'uint16',
          'device': 'uint16',
          'os': 'uint16',
          'channel': 'uint16',
          'is_attributed': 'uint8'}
test = pd.read_csv(test_dir, dtype=dtypes, parse_dates=['click_time', 'attributed_time'])